## Install Selenium

In [ ]:
import importlib
try:
    importlib.import_module('selenium')
    print('selenium is already installed')
except:
    !pip install scrapy
    print('selenium is now installed')
try:
    importlib.import_module('anticaptchaofficial')
    print('anticaptchaofficial is already installed')
except:
    !pip install anticaptchaofficial
    print('anticaptchaofficial is now installed')
try:
    importlib.import_module('requests')
    print('requests is already installed')
except:
    !pip install requests
    print('requests is now installed')

# Import Necessary Modules

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException, UnexpectedAlertPresentException, NoAlertPresentException
from anticaptchaofficial.recaptchav3proxyless import *
from time import sleep
from random import randint

## Initialize the webdriver

In [ ]:
options = webdriver.FirefoxOptions()
options.add_argument("--private-window")
driver = webdriver.Firefox(options=options)
driver.maximize_window()

## Open the Superkicks website

In [ ]:
def redirect(URL):
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    header = wait.until(EC.presence_of_element_located((By.ID, 'MegaMenu-Content-2')))
    return wait

# Login the Acoount

In [ ]:
def login():
    driver.get('https://www.superkicks.in/account/login')
    login_form = driver.find_element(By.ID, 'customer_login')
    fields = login_form.find_elements(By.CLASS_NAME, 'field')
    try:
        sleep(2)
        fields[0].find_element(By.TAG_NAME, 'input').send_keys(USER_NAME)
    except ElementClickInterceptedException:
        close_add()
        driver.switch_to.default_content()
        sleep(2)
        fields[0].find_element(By.TAG_NAME, 'input').send_keys(USER_NAME)
    try:
        sleep(3)
        fields[1].find_element(By.TAG_NAME, 'input').send_keys(PASSWORD)
    except ElementClickInterceptedException:
        close_add()
        driver.switch_to.default_content()
        sleep(3)
        fields[1].find_element(By.TAG_NAME, 'input').send_keys(PASSWORD)
    button = driver.find_element(By.CLASS_NAME, 'button--full-width')
    try:
        sleep(2)
        button.click()
    except ElementClickInterceptedException:
        login()
    sleep(2)

## Bypass recaptcha

In [ ]:
def send_token(g_response):
    driver.execute_script("document.getElementById('g-recaptcha-response').value ='{}';".format(g_response))
    driver.find_element(By.XPATH, '/html/body/main/div/form/input[2]').click()
        
def bypass():
    solver = recaptchaV3Proxyless()
    solver.set_verbose(1)
    solver.set_key("Enter Site Key")
    solver.set_website_url("Enter Page URL where Recaptcha exists")
    solver.set_website_key("Enter the Site Key of that Webpage")
    solver.set_page_action("home_page")
    solver.set_min_score(0.9)

    solver.set_soft_id(0)
    g_response = solver.solve_and_return_solution()
    sleep(2)
    try:
        if g_response != 0:
            print("g-response: "+g_response)
        else:
            print("task finished with error "+solver.error_code)
            send_token(g_response)
    except:
        return g_response

# Place the Order

In [ ]:
def extract_labels():
    try:
        fieldset = driver.find_element(By.XPATH, '/html/body/main/section[1]/section/div/div[2]/div/variant-radios/fieldset')
        labels = fieldset.find_elements(By.CLASS_NAME, 'InStock')
    except:
        labels = Select(driver.find_element(By.ID, 'Variants-template--16732352872699__main'))
        labels = labels.options
    return labels

def add_to_cart():
    try:
        product_form = driver.find_element(By.TAG_NAME, 'product-form').find_element(By.TAG_NAME, 'form')
        add_to_cart = product_form.find_element(By.CLASS_NAME, 'product-form__buttons').find_element(By.TAG_NAME, 'button')
        add_to_cart.click()
    except ElementClickInterceptedException:
        driver.refresh()
        add_to_cart()

def check_out(q):
    try:
        input_label = driver.find_element(By.ID, 'Drawer-quantity-1')
        input_label.clear()
        input_label.send_keys(q)
        button = driver.find_element(By.ID, 'CartDrawer-Checkout')
        button.click()
    except:
        driver.refresh()
        driver.get(driver.find_element(By.ID, 'cart-icon-bubble').get_attribute('href'))
        anchor = driver.find_element(By.XPATH, '/html/body/main/div/cart-items/div[1]/form/div/div/table/tbody/tr/td[4]/div[1]/cart-remove-button/a')
        driver.get(anchor.get_attribute('href'))
        sleep(2)
        driver.get(URL)
        place_order()

def shipping():
    try:
        footer = driver.find_element(By.TAG_NAME, 'form').find_element(By.CLASS_NAME, 'step__footer')
        shipping_info()
        button = footer.find_element(By.TAG_NAME, 'button')
        button.click()
        return ''
    except ElementClickInterceptedException:
        refresh()
        shipping()
    except:
        return 'No'

def shipping_info():
    driver.find_element(By.ID, 'checkout_email').send_keys(USER_NAME)
    driver.find_element(By.ID, 'checkout_shipping_address_first_name').send_keys(FIRST_NAME)
    driver.find_element(By.ID, 'checkout_shipping_address_last_name').send_keys(LAST_NAME)
    driver.find_element(By.ID, 'checkout_shipping_address_address1').send_keys(ADDRESS)
    driver.find_element(By.ID, 'checkout_shipping_address_address2').send_keys(APARTMENT)
    driver.find_element(By.ID, 'checkout_shipping_address_city').send_keys(CITY)
    dropdown = driver.find_element(By.ID, 'checkout_shipping_address_province')
    select = Select(dropdown)
    select.select_by_visible_text(STATE)
    driver.find_element(By.ID, 'checkout_shipping_address_zip').send_keys(PIN_CODE)
    driver.find_element(By.ID, 'checkout_shipping_address_phone').send_keys(PHONE_NO)

def place_order(default=True):
    try:
        labels = extract_labels()
    except:
        driver.get(URL)
        labels = extract_labels()
        print('Error occured before label')
    i = 0
    while i < len(labels):
        if labels[i].text in SIZE:
            try:
                labels[i].click()
            except:
                driver.get(URL)
                labels = extract_labels()
                print('Error occured after if statement')
            labels[i].click()
            i += 1
            sleep(3)
            add_to_cart()
            sleep(5)
            try:
                check_out(QUANTITY[0])
                print('Quantity = ', QUANTITY[0])
                QUANTITY.pop(0)
                print('after pop = ', QUANTITY)
                sleep(1)
                try:
                    if shipping() == 'No':
                        print('This Product has been Sold --> Out of Stock')
                        break
                except:
                    pass
            except StaleElementReferenceException or NoSuchElementException:
                pass
            except ElementClickInterceptedException:                
                place_order(False)
            paymant()
            print('Your Order has been placed')
            driver.get(URL)
            labels = extract_labels()
        else:
            i += 1

## Pay the Amount

In [ ]:
def paymant():
    button = driver.find_element(By.ID, 'continue_button')
    button.click()
    cashfree_payment = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/main/div[1]/div/form/div[1]/div[2]/div[1]/fieldset/div[3]/div[2]/label')
    cashfree_payment.click()
    pay_button = driver.find_element(By.XPATH, '//*[@id="continue_button"]')
    pay_button.click()
    skip = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dkp-container-main"]/div[3]/div/div[2]/div/div[2]/div[1]/button')))
    skip.click()
    payment_methods = driver.find_element(By.XPATH, '//*[@id="dkp-container-main"]/div[3]/div[1]').find_elements(By.TAG_NAME, 'h2')
    for method in payment_methods:
        if method.text == 'Card':
            method.find_element(By.XPATH, '..').find_element(By.XPATH, '..').click()
            
            card_no = 4160210836812787
            card_holder_name = 'raunka agarwal'
            date = '02/27'
            cvv = 451
            
            driver.find_element(By.ID, 'CardNumber1').send_keys(card_no)
            driver.find_element(By.ID, 'CardHolderName1').send_keys(card_holder_name)
            driver.find_element(By.ID, 'CardDate1').send_keys(date)
            driver.find_element(By.ID, 'CVVFormatter1').send_keys(cvv)
            
            sleep(2)
            pay = driver.find_element(By.XPATH, '//*[@id="dkp-container-main"]/div[3]/div[2]/div[2]/div/div/form/div[9]/div/button')
            pay.click()
            submit = input("Have you entered the OTP and clicked on the Submit button, y/n? ")
            return

## Start the Bot

In [ ]:
if __name__ == "__main__":
    URL = input('Enter the URL of Product= ')
    
    SIZE = input("Enter the Size you want to purchase: ") + ' '
    SIZE = SIZE.split(' ')
    
    QUANTITY = input('Enter the quantity you want to purchase: ') + ' '
    QUANTITY = QUANTITY.split(' ')
    print(QUANTITY)
    
    wait = redirect(URL)
    USER_NAME = 'abc@gmail.com'
    PASSWORD = '*********'
    PHONE_NO = '+923000000000'
    ADDRESS = ""
    APARTMENT = ''
    CITY = ''
    STATE = ''
    PIN_CODE = 110065
    COUNTRY = ""
    FIRST_NAME = ''
    LAST_NAME = ''

    place_order()